In [107]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import col, lower, upper, trim, to_date, expr, lit, exp, decode, when, date_add,min, max
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import datetime
import json
import configparser as cp
from datetime import date
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

In [108]:
input_date = date.today()
udf3 = udf(derive_prev_date, DateType())
tps_property_interested_columns = ["global_property_id", "status", "postponement_notification_date", "cancel_notification_date", "is_scheduled", "sold_status_desc","removal_date", "source_seller", 'bta_status',
                            "file_received_date", "property_State", "property_county", "net_sold_status", "sold_status", "winning_bid", "is_deleted", "is_scheduled", "source_seller_id", "venue_id", "auction_id",
                            "is_upset_bid_confirmed", "service_type"]
tps_property_df = spark.read.parquet("s3://adc-dev/data/derived/dw/tps_property/part-00000*").select([col(c) for c in tps_property_interested_columns])
seller_df = spark.read.parquet("s3://adc-dev/data/derived/dw/seller/")
event_venue_df = spark.read.parquet("s3://adc-dev/data/derived/dw/event_venue/")
output = tps_property_df.join(seller_df, ['source_seller_id', 'source_seller'], 'left_outer') \
                        .join(event_venue_df, ((tps_property_df['auction_id'] == event_venue_df['auction_id']) &
                        ((tps_property_df['venue_id'] == event_venue_df['venue_id']) | (tps_property_df.venue_id.isNull() & event_venue_df.venue_id.isNull()))), 'left_outer')
output = output.filter((col('is_deleted') == False) & (col('is_scheduled') == True) & (~ upper(col('event_code')).isin(['E-900X', 'E-900Z'])) & (col('non_billable_flag') == False) & \
                (col('auction_date') >= date_add(lit(input_date), -40)))
assetlevel_interested_columns = ['global_property_id', 'auction_date', 'seller_classification', 'Servicer', 'status', 'postponement_notification_date', 'cancel_notification_date', 'event_code',
                            'is_scheduled', 'sold_status_desc', 'removal_date', 'file_received_date', 'property_State', 'property_county', 'bta_status', 'net_sold_status', 'sold_status', 'winning_bid',
                            'auction_date', 'non_billable_flag', 'is_upset_bid_confirmed', 'service_type']
output = output.select([col(c) for c in assetlevel_interested_columns])
Assetlevel = output.withColumn('globalpropertyid', col('global_property_id'))\
                .withColumn('AuctionDate', col('auction_date')) \
                .withColumn('IsPostponed', when((col('status') == 'Postponed'), True).otherwise(False)) \
                .withColumn('PostponementNotificationDate', col('postponement_notification_date')) \
                .withColumn('IsCancelled', when((col('status') == 'Cancelled'), True).otherwise(False)) \
                .withColumn('cancelnotificationdate', col('cancel_notification_date')) \
                .withColumn('auctioncode', col('event_code')) \
                .withColumn('scheduled', col('is_scheduled')) \
                .withColumn('Active_Status', when(col('sold_status_desc').isin(['Not Applicable','Unknown']), col('status')).otherwise(col('sold_status_desc'))) \
                .withColumn('removaldate', col('removal_date')) \
                .withColumn('auctionfilereceiveddate', col('file_received_date')) \
                .withColumn('propertystate', col('property_State')) \
                .withColumn('propertycounty', col('property_county')) \
                .withColumn('isauction', when((col('bta_status') == 'BTA'), True).otherwise(False)) \
                .withColumn('isauction2', when((col('bta_status') == 'BTA') & (col('bta_status') != 'BTA Pending'), True).otherwise(False)) \
                .withColumn('isauctionpending', when((col('bta_status') == 'BTA Pending'), True).otherwise(False)) \
                .withColumn('issold', when((col('bta_status') == 'Net Sold'), True).otherwise(False)) \
                .withColumn('issold_test', when((col('bta_status') == 'Sold'), True).otherwise(False)) \
                .withColumn('winningbid', col('winning_bid')) \
                .withColumn('Sched_Count', col('is_scheduled')) \
                .withColumn('AuctionOrderStatus', when(col('global_property_id').isNull(), lit('New')).otherwise(lit('Churn'))) \
                .withColumn('History', when((col('auction_date')>=date_add(lit(input_date), -40)) & (col('auction_date') <= lit(input_date)), 1).otherwise(0)) \
                .withColumn('Future', when((col('auction_date')>=lit(input_date)), 1).otherwise(0)) \
                .withColumn('prev_date', udf3(output.status, output.postponement_notification_date, output.cancel_notification_date, output.auction_date)) \
                .withColumn('Pending', when((col('property_State') == 'NC') & (col('net_sold_status') != 'Net Sold') & (col('is_upset_bid_confirmed') == False), 1).otherwise(0))
Assetlevel.show(2)

+------------------+------------+---------------------+-----------+-------+------------------------------+------------------------+----------+------------+----------------+------------+------------------+--------------+---------------+-----------+---------------+-----------+-----------+------------+-----------------+----------------------+-------------+----------------+-----------+-----------+----------------------------+-----------+----------------------+-----------+---------+-------------+-----------+-----------------------+-------------+--------------+---------+----------+----------------+------+-----------+----------+-----------+------------------+-------+------+----------+-------+
|global_property_id|auction_date|seller_classification|   Servicer| status|postponement_notification_date|cancel_notification_date|event_code|is_scheduled|sold_status_desc|removal_date|file_received_date|property_State|property_county| bta_status|net_sold_status|sold_status|winning_bid|auction_date|non_b

In [47]:
def derive_prev_date(status, postponement_notification_date, cancel_notification_date, auction_date):
    if status == 'Postponed' and postponement_notification_date is not None:
        return postponement_notification_date
    elif status == 'Cancelled' and postponement_notification_date is not None:
        return cancel_notification_date
    else:
        return auction_date

In [109]:
prev_date = Assetlevel.select(['globalpropertyid', 'AuctionDate', 'prev_date']).dropDuplicates()
prev_date.show(10)

+----------------+-----------+----------+
|globalpropertyid|AuctionDate| prev_date|
+----------------+-----------+----------+
|         3103475| 2020-06-27|2020-06-27|
|         3092573| 2020-05-20|2010-01-01|
|         3105495| 2020-05-28|2020-05-28|
|         3098410| 2020-06-28|2020-06-28|
|         3101817| 2020-05-22|2020-05-22|
|         3105561| 2020-05-28|2020-05-28|
|         3101406| 2020-06-30|2020-06-30|
|         3100820| 2020-05-23|2020-05-23|
|         3098838| 2020-05-21|2020-05-21|
|         3102063| 2020-06-26|2020-06-26|
+----------------+-----------+----------+
only showing top 10 rows



In [119]:
insert_date = Assetlevel.select(['globalpropertyid', 'auctioncode', 'auctionfilereceiveddate'])\
        .groupby(['globalpropertyid', 'auctioncode'])\
         .agg(min(col("auctionfilereceiveddate")).alias('first_receipt_dt'), min("auctionfilereceiveddate").alias('insert_date'))\
        .withColumnRenamed("globalpropertyid","global_property_id")\
        .withColumnRenamed("auctioncode", "auction_code")
insert_date.show(10)

+------------------+------------+----------------+-----------+
|global_property_id|auction_code|first_receipt_dt|insert_date|
+------------------+------------+----------------+-----------+
|           3100573|      E-WO67|      2014-06-08| 2014-06-08|
|           3097247|      E-JMQR|      2020-05-14| 2020-05-14|
|           3102837|      E-43FX|      2014-06-08| 2014-06-08|
|           3102522|      E-TROG|      2014-06-08| 2014-06-08|
|           3092077|    E-15349A|      2019-04-08| 2019-04-08|
|           3105640|      E-VR4U|      2014-06-08| 2014-06-08|
|           3105561|      E-NRDK|      2014-06-08| 2014-06-08|
|           3102815|      E-LC74|      2014-06-08| 2014-06-08|
|           3092129|    E-15376A|      2020-04-06| 2020-04-06|
|           3096086|      E-3CY7|      2020-04-30| 2020-04-30|
+------------------+------------+----------------+-----------+
only showing top 10 rows



In [121]:
hist_deep_dive = Assetlevel.join(prev_date, 'GlobalPropertyId', 'left')\
                            .join(insert_date, (Assetlevel.globalpropertyid == insert_date.global_property_id) & (Assetlevel.auctioncode==insert_date.auction_code), 'left')
hist_deep_dive = hist_deep_dive.filter(col('History') == 1)
hist_deep_dive = hist_deep_dive.withColumn('AuctionOrderStatus',  when(col('global_property_id').isNull(), lit('New')).otherwise(lit('Churn')))
hist_deep_dive = 
hist_deep_dive.show(10)

+----------------+------------------+------------+---------------------+-----------+---------+------------------------------+------------------------+----------+------------+----------------+------------+------------------+--------------+---------------+-----------+---------------+--------------+-----------+------------+-----------------+----------------------+-------------+-----------+-----------+----------------------------+-----------+----------------------+-----------+---------+-------------+-----------+-----------------------+-------------+--------------+---------+----------+----------------+------+-----------+----------+-----------+------------------+-------+------+----------+-------+-----------+----------+------------------+------------+----------------+-----------+
|globalpropertyid|global_property_id|auction_date|seller_classification|   Servicer|   status|postponement_notification_date|cancel_notification_date|event_code|is_scheduled|sold_status_desc|removal_date|file_receive

In [458]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from datetime import datetime


schema = StructType([
    StructField("holiday_date", DateType()),
    StructField("holiday_name", StringType())
])

raw_holidays = spark.read.format("csv").schema(schema).load("s3://adc-dev/data/raw/dw/holidays/holidays.csv")
raw_holidays.show(10)
date_fmt = "%Y-%m-%d"
min_date = '2000-01-01'
max_date = '2049-12-31'
a = datetime.strptime(min_date, date_fmt)
b = datetime.strptime(max_date, date_fmt)
sql = "select posexplode(split(repeat(',', {}), ','))".format(delta.days)
print(sql)
a=spark.sql(sql).withColumn('startdate', lit(min_date).cast(DateType()))
a=a.withColumn('date_code', F.expr("""date_add(startdate ,pos)""")).drop(*['col', 'pos', 'startdate'])
a = a.withColumn('date_key', date_format(a.date_code, 'yyyyMMdd')) \
    .withColumn('YearCode', year(a.date_code)) \
    .withColumn('QuarterCode', quarter(a.date_code)) \
    .withColumn('MonthCode', month(a.date_code)) \
    .withColumn('week_code', date_format(a.date_code, 'w')) \
    .withColumn('day_of_week', dayofweek(a.date_code)) \
    .withColumn('day_of_month', dayofmonth(a.date_code)) \
    .withColumn('week_of_month', date_format(a.date_code, 'F')) \
    .withColumn('month_name', date_format(a.date_code, 'MMMM')) \
    .withColumn('day_name', date_format(a.date_code, 'EEEE')) \
    .withColumn('month_name_short', date_format(a.date_code, 'MMM')) \
    .withColumn('day_name_short', date_format(a.date_code, 'EEE')) \
    .withColumn('is_weekend', when(date_format(a.date_code, 'EEE').isin(["Sat", "Sun"]), True).otherwise(False)) \
    .withColumn('end_of_month', last_day(a.date_code))\
    .withColumn('IsEOM', when(last_day(a.date_code) == a.date_code, True).otherwise(False))\
    .withColumn('previous_eom', last_day(add_months(a.date_code, -1))) \
    .withColumn('previous_year_eom', last_day(add_months(a.date_code, -12))) \
    .withColumn('next_eom', last_day(add_months(a.date_code, 1))) \
    .withColumn('FirstDayOfWeek', F.expr("""date_add(date_code ,-1*dayofweek(date_code)+1)""")) \
    .withColumn('FirstDayOfMonth', F.expr("""date_add(date_code ,-1*dayofmonth(date_code)+1)""")) \
    .withColumn('FirstDayOfYear', F.expr("""date_add(date_code ,-1*date_format(date_code, 'D')+1)""")) \
    .withColumn('FirstDayOfPreviousWeek', F.expr("""date_add(date_code ,-1*dayofweek(date_code)-6)""")) \
    .withColumn('YYYYMMDD', date_format(a.date_code, 'yyyyMMdd')) \
    .withColumn('MMDDYYYY', date_format(a.date_code, 'MMddyyyy')) \
    .withColumn('DDMMYYYY', date_format(a.date_code, 'ddMMyyyy')) \
    .withColumn('YYYYMM', date_format(a.date_code, 'yyyyMM')) \
    .withColumn('MMYYYY', date_format(a.date_code, 'MMyyyy')) \
    .withColumn('YYYYQQ', year(a.date_code)*100+quarter(a.date_code)) \
    .withColumn('QQYYYY', lpad(quarter(a.date_code)*10000+year(a.date_code), 6, '0')) \
    .withColumn('YearQuarter', concat(year(a.date_code), lit('Q'), quarter(a.date_code))) \
    .withColumn('QuarterYear', concat(lit('Q'), quarter(a.date_code), year(a.date_code)))

a = a.withColumn('FirstDayOfPreviousMonth', F.expr("""date_add(last_day(add_months(date_code, -2)),1)""")) \
    .withColumn('FirstDayOfPreviousYear', F.expr("""add_months(FirstDayOfYear, -12)"""))

a = a.join(raw_holidays, raw_holidays['holiday_date'] == a['date_code'], 'left')
a = a.withColumn('IsHoliday', when(a.holiday_date.isNull(), False).otherwise(True)) \
    .withColumn('IsBusinessDay', when((a.holiday_date.isNull()) & (a.is_weekend == False) & (~ a.YearCode.isin(1900, 2999)), True).otherwise(False))
a.registerTempTable("date_staging")
a = spark.sql("""select *,
        sum(case when IsBusinessDay == True then 1 else 0 end) over (partition by YYYYMM order by date_key rows between unbounded preceding and current row) as BusinessDaysMonthToDate,
        sum(case when IsBusinessDay == True then 1 else 0 end) over (partition by YYYYMM order by date_key rows between 1 following and unbounded following ) as BusinessDaysRemaining,
        sum(case when IsBusinessDay == True then 1 else 0 end) over (partition by YYYYQQ order by date_key rows between unbounded preceding and current row) as BusinessDaysQuarterToDate,
        sum(case when IsBusinessDay == True then 1 else 0 end) over (partition by YearCode order by date_key rows between unbounded preceding and current row) as BusinessDaysYearToDate,
        max(case when IsBusinessDay == True then date_code else Null end) over (order by date_key rows between unbounded preceding and 1 preceding) as PreviousBusinessDay,
        min(case when IsBusinessDay == True then date_code else Null end) over (order by date_key rows between 1 following  and unbounded following) as NextBusinessDay
        from date_staging order by date_key""")
a.write.format("parquet").mode('overwrite').option("path","s3://adc-dev/data/derived/dw/date").saveAsTable('default.dw_date')


+------------+--------------------+
|holiday_date|        holiday_name|
+------------+--------------------+
|  2000-01-01|      New Year's Day|
|  2000-01-17|Martin Luther Kin...|
|  2000-02-21|Washington's Birt...|
|  2000-05-29|        Memorial Day|
|  2000-07-04|    Independence Day|
|  2000-09-04|           Labor Day|
|  2000-10-10|Columbus Day (Tra...|
|  2000-11-11|        Veterans Day|
|  2000-11-23|    Thanksgiving Day|
|  2000-12-25|       Christmas Day|
+------------+--------------------+
only showing top 10 rows

select posexplode(split(repeat(',', 18262), ','))


In [485]:
def get_interested_columns_df(df):
    interested_col_mapping = dict(zip(["businessdaysMonthToDate",
                "businessdaysQuarterToDate",
                "businessdaysYearToDate",
                "businessdaysRemaining",
                "date_code",
                "date_key",
                "day_name",
                "day_name_short",
                "day_of_month",
                "day_of_week",
                "DDmmyyyy",
                "end_of_month",
                "firstdayofmonth",
                "firstdayofpreviousmonth",
                "firstdayofpreviousweek",
                "firstdayofpreviousyear",
                "firstdayofweek",
                "firstdayofyear",
                "holiday_name",
                "isbusinessday",
                "iseom",
                "isholiday",
                "is_weekend",
                "mmddyyyy",
                "mmyyyy",
                "monthcode",
                "month_name",
                "month_name_short",
                "nextbusinessday",
                "next_eom",
                #"next15businessday",
                #"next2businessday",
                "previousbusinessday",
                "previous_eom",
                #"previousmonthbusinessday",
                #"previousquarterbusinessday",
                #"previous_3_months_business_day",
                #"previous_12_months_business_day",
                #"previousyearbusinessday",
                "previous_year_eom",
                "qqyyyy",
                "quartercode",
                "quarteryear",
                "week_code",
                "week_of_month",
                "yearcode",
                "yearquarter",
                "yyyymm",
                "yyyymmdd",
                "yyyyqq"],["business_days_month_to_date",
                "business_days_quarter_to_date",
                "business_days_year_to_date",
                "business_days_remaining",
                "date_code",
                "date_key",
                "day_name",
                "day_name_short",
                "day_of_month",
                "day_of_week",
                "DDmmyyyy",
                "end_of_month",
                "first_day_of_month",
                "first_day_of_previous_month",
                "first_day_of_previous_week",
                "first_day_of_previous_year",
                "first_day_of_week",
                "first_day_of_year",
                "holiday_name",
                "is_business_day",
                "is_eom",
                "is_holiday",
                "is_weekend",
                "mmddyyyy",
                "mmyyyy",
                "month_code",
                "month_name",
                "month_name_short",
                "next_business_day",
                "next_eom",
                #"next15_business_day",
                #"next2_business_day",
                "previous_business_day",
                "previous_eom",
                #"previous_month_business_day",
                #"previous_quarter_business_day",
                #"previous_3_months_business_day",
                #"previous_12_months_business_day",
                #"previous_year_business_day",
                "previous_year_eom",
                "qqyyyy",
                "quarter_code",
                "quarter_year",
                "week_code",
                "week_of_month",
                "year_code",
                "year_quarter",
                "yyyymm",
                "yyyymmdd",
                "yyyyqq"]))
    df = df.select([col(c).alias(interested_col_mapping.get(c, c)) for c in interested_col_mapping.keys()])
    df.show(10)


In [403]:
df = spark.createDataFrame(
    [("11/25/1991",), ("11/24/1991",), ("11/30/1991",)], 
    ['date_str']
)

In [486]:
get_interested_columns_df(a)

+---------------------------+-----------------------------+--------------------------+-----------------------+----------+--------+---------+--------------+------------+-----------+--------+------------+------------------+---------------------------+--------------------------+--------------------------+-----------------+-----------------+--------------+---------------+------+----------+----------+--------+------+----------+----------+----------------+-----------------+----------+---------------------+------------+-----------------+------+------------+------------+---------+-------------+---------+------------+------+--------+------+
|business_days_month_to_date|business_days_quarter_to_date|business_days_year_to_date|business_days_remaining| date_code|date_key| day_name|day_name_short|day_of_month|day_of_week|DDmmyyyy|end_of_month|first_day_of_month|first_day_of_previous_month|first_day_of_previous_week|first_day_of_previous_year|first_day_of_week|first_day_of_year|  holiday_name|is_bus

In [410]:
schema = StructType([
    StructField("holiday_date", DateType()),
    StructField("holiday_name", StringType())
])

raw_holidays = spark.read.format("csv").schema(schema).load("s3://adc-dev/data/raw/dw/holidays/holidays.csv")
raw_holidays.show(10)
raw_holidays.printSchema()

+------------+--------------------+
|holiday_date|        holiday_name|
+------------+--------------------+
|  2000-01-01|      New Year's Day|
|  2000-01-17|Martin Luther Kin...|
|  2000-02-21|Washington's Birt...|
|  2000-05-29|        Memorial Day|
|  2000-07-04|    Independence Day|
|  2000-09-04|           Labor Day|
|  2000-10-10|Columbus Day (Tra...|
|  2000-11-11|        Veterans Day|
|  2000-11-23|    Thanksgiving Day|
|  2000-12-25|       Christmas Day|
+------------+--------------------+
only showing top 10 rows

root
 |-- holiday_date: date (nullable = true)
 |-- holiday_name: string (nullable = true)

